In [91]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

### Obtener Datos Financieros

In [92]:
def get_profile(ticker):
    """Obtiene y analiza la información del perfil de una acción dada desde Yahoo Finance.
    """
    # Construir la URL de la página de la declaración de ingresos
    url= f"https://finance.yahoo.com/quote/{ticker}/profile"

    # Definir encabezados para imitar una solicitud de navegador
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

    # Realizar solicitud GET a la URL
    response = requests.get(url, headers=headers)

    # Analizar el contenido HTML usando BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraer sector y industria
    text = [link.get_text().strip() for link in soup.find_all('a', class_="subtle-link fin-size-large svelte-wdkn18")]
    
    #Definir titulos de las columnas para hacer el DataFrame
    titles = ['Sector','Industry']

    # Crear un DataFrame con los datos extraídos
    df = pd.DataFrame([text], columns = titles)
    df['Ticker'] = ticker
    
    return df


In [93]:
def get_income_statement(ticker):
    """Obtiene y analiza los datos de la declaración de ingresos para un símbolo de acción dado de Yahoo Finance.
    """
    # Construir la URL de la página de la declaración de ingresos
    url= f"https://finance.yahoo.com/quote/{ticker}/financials"

    # Definir encabezados para imitar una solicitud de navegador
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

    # Realizar solicitud GET a la URL
    response = requests.get(url, headers=headers)

    # Analizar el contenido HTML usando BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extraer encabezados de columnas
    find1 = soup.find_all('div', class_="column svelte-1ezv2n5 alt")
    find2 = soup.find_all('div', class_="column svelte-1ezv2n5")
    group = [x.text.strip() for x in find1 + find2]

    # Ordenar los encabezados según el orden especificado
    if len(group) == 6:
        order = [0, 3, 1, 4, 2, 5]
    if len(group) == 5:
        order = [0, 3, 1, 4, 2]

    titles = [group[i] for i in order]

    # Extraer datos de encabezados
    encabezados = [link.get_text().strip() for link in soup.find_all('div', class_="column sticky svelte-1xjz32c")]

    # Extraer datos de columnas
    row1 = [link.get_text().strip() for link in soup.find_all('div', class_="column svelte-1xjz32c alt")]
    row2 = [link.get_text().strip() for link in soup.find_all('div', class_="column svelte-1xjz32c")]

    # Agrupar los elementos en sublistas
    sub_listas_a = [row1[i:i+3] for i in range(0, len(row1), 3)]
    sub_listas_b = [row2[i:i+(3 if len(titles) == 6 else 2)] for i in range(0, len(row2), (3 if len(titles) == 6 else 2))]
    # Combinar sublistas
    combinada = [sublist_a + sublist_b for sublist_a, sublist_b in zip(sub_listas_a, sub_listas_b)]
    # Ordenar según el orden especificado y convertir a float
    data = [[float(sublist[i].replace(",", "")) if sublist[i] != "--" else 0 for i in order] for sublist in combinada]

    # Crear el diccionario
    dic = dict(zip(encabezados, data))
    df = pd.DataFrame(dic)
    df.index = titles
    return df


In [94]:
def get_balance_sheet(ticker):
    """Obtiene y analiza los datos de la declaración de ingresos para un símbolo de acción dado de Yahoo Finance.
    """
    # Construir la URL de la página de la declaración de ingresos
    url= f"https://finance.yahoo.com/quote/{ticker}/balance-sheet"

    # Definir encabezados para imitar una solicitud de navegador
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

    # Realizar solicitud GET a la URL
    response = requests.get(url, headers=headers)

    # Analizar el contenido HTML usando BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extraer encabezados de columnas
    find1 = soup.find_all('div', class_="column svelte-1ezv2n5 alt")
    find2 = soup.find_all('div', class_="column svelte-1ezv2n5")
    group = [x.text.strip() for x in find1 + find2]

    # Ordenar los encabezados según el orden especificado y por cantidad de columnas
    if len(group) == 5:
        order = [0, 3, 1, 4, 2]
    if len(group) == 4:
        order = [0, 2, 1, 3]
    titles = [group[i] for i in order]

    # Extraer datos de encabezados
    encabezados = [link.get_text().strip() for link in soup.find_all('div', class_="column sticky svelte-1xjz32c")]

    # Extraer datos de columnas
    row1 = [link.get_text().strip() for link in soup.find_all('div', class_="column svelte-1xjz32c alt")]
    row2 = [link.get_text().strip() for link in soup.find_all('div', class_="column svelte-1xjz32c")]


    # Agrupar los elementos en sublistas
    sub_listas_a = [row1[i:i+(3 if len(titles) == 5 else 2)] for i in range(0, len(row1), (3 if len(titles) == 5 else 2))]
    sub_listas_b = [row2[i:i+2] for i in range(0, len(row2), 2)]

    # Combinar sublistas
    combinada = [sublist_a + sublist_b for sublist_a, sublist_b in zip(sub_listas_a, sub_listas_b)]

    # Ordenar según el orden especificado y convertir a float
    data = [[float(sublist[i].replace(",", "")) if sublist[i] != "--" else 0 for i in order] for sublist in combinada]

    # Crear el diccionario
    dic = dict(zip(encabezados, data))
    df = pd.DataFrame(dic)
    df.index = titles
    return df


In [95]:
def get_cash_flow(ticker):
    """Obtiene y analiza los datos de la declaración de ingresos para un símbolo de acción dado de Yahoo Finance.
    """
    # Construir la URL de la página de la declaración de ingresos
    url= f"https://finance.yahoo.com/quote/{ticker}/cash-flow"

    # Definir encabezados para imitar una solicitud de navegador
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

    # Realizar solicitud GET a la URL
    response = requests.get(url, headers=headers)

    # Analizar el contenido HTML usando BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extraer encabezados de columnas
    find1 = soup.find_all('div', class_="column svelte-1ezv2n5 alt")
    find2 = soup.find_all('div', class_="column svelte-1ezv2n5")
    group = [x.text.strip() for x in find1 + find2]

    # Ordenar los encabezados según el orden especificado y por cantidad de columnas
    if len(group) == 6:
        order = [0, 3, 1, 4, 2, 5]
    if len(group) == 5:
        order = [0, 3, 1, 4, 2]
    titles = [group[i] for i in order]

    # Extraer datos de encabezados
    encabezados = [link.get_text().strip() for link in soup.find_all('div', class_="column sticky svelte-1xjz32c")]

    # Extraer datos de columnas
    row1 = [link.get_text().strip() for link in soup.find_all('div', class_="column svelte-1xjz32c alt")]
    row2 = [link.get_text().strip() for link in soup.find_all('div', class_="column svelte-1xjz32c")]

    # Agrupar los elementos en sublistas
    sub_listas_a = [row1[i:i+3] for i in range(0, len(row1), 3)]
    sub_listas_b = [row2[i:i+(3 if len(titles) == 6 else 2)] for i in range(0, len(row2), (3 if len(titles) == 6 else 2))]

    # Combinar sublistas
    combinada = [sublist_a + sublist_b for sublist_a, sublist_b in zip(sub_listas_a, sub_listas_b)]

    # Ordenar según el orden especificado y convertir a float
    data = [[float(sublist[i].replace(",", "")) if sublist[i] != "--" else 0 for i in order] for sublist in combinada]

    # Crear el diccionario
    dic = dict(zip(encabezados, data))
    df = pd.DataFrame(dic)
    df.index = titles
    return df


In [96]:
def get_price(ticker):
    """Obtiene los datos históricos de precios de una acción desde Yahoo Finance.
    """
    # Obtener los datos históricos de precios de la acción
    df = yf.Ticker(ticker).history(start='2021-01-01', end=datetime.now().strftime('%Y-%m-%d'))
    
    # Seleccionar la columna de precios y reiniciar el índice
    df = df.iloc[:, 0].reset_index()
    
    # Formatear la columna de fechas y renombrar la columna de precios
    df["Date"] = df["Date"].dt.strftime('%d/%m/%Y')
    df.rename(columns={"Open": "Price"}, inplace=True)
    
    # Agregar la columna de símbolo de la acción
    df["Ticker"] = ticker
    
    return df


In [97]:
def selections(ticker):
    # Obtener los estados financieros
    df_s = get_income_statement(ticker)  # Estado de ingresos
    df_b = get_balance_sheet(ticker)     # Balance general
    df_cf = get_cash_flow(ticker)         # Flujo de efectivo
    
    # Lista de las filas que deseas seleccionar de los estados financieros
    select_statement = ['Total Revenue', 'Operating Income', 'Net Income Common Stockholders']
    # Filtrar el DataFrame de estados financieros utilizando la lista select_statement
    statement = df_s[select_statement]
    
    # Lista de las filas que deseas seleccionar del balance
    select_balance = ['Total Debt', 'Net Debt', 'Share Issued', 'Common Stock Equity']
    
    # Verifica si todas las columnas de select_balance están presentes en el DataFrame de balance
    for col in select_balance:
        if col not in df_b.columns:
            df_b[col] = pd.NA

    # Filtrar el DataFrame de balance utilizando la lista select_balance
    balance = df_b[select_balance]

    # Lista de las filas que deseas seleccionar del flujo de efectivo
    select_cashflow = ['Free Cash Flow']
    # Filtrar el DataFrame de flujo de efectivo utilizando la lista select_cashflow
    cashflow = df_cf[select_cashflow]
    
    # Concatenar las filas seleccionadas
    select = pd.concat([statement, balance, cashflow], axis=1).iloc[1:].reset_index()
    
    # Agregar una columna con el ticker de la acción
    select['Ticker'] = ticker
    
    # Renombrar columnas para mayor claridad
    select = select.rename(columns={"index": "Date", "Total Revenue": "Revenue", "Net Income Common Stockholders": "Net Income", "Operating Income": "EBIT", "Common Stock Equity": "Equity"})
    
    return select

In [98]:
# Símbolos de las acciones a procesar
list_stock = list_stock = ['ADBE','MSFT','AAPL','QCOM','AMD','NVDA','SHOP','CRM','ABNB','BKNG','LULU','MBUU','NKE','ADS.DE','V','MA','PYPL','GRBK','LEN','RACE','TSLA','AMZN','SBUX','MCD','KO','PEP','WMT','COST','TGT','NFLX','DIS','META','GOOGL']


### Procesar Precios de Acciones

In [99]:
# Definir una función para procesar el DataFrame de precios de acciones
def process_stock_prices(dataframe):
    dataframe['Date'] = pd.to_datetime(dataframe['Date'], format='%d/%m/%Y')
    dataframe['Price'] = dataframe['Price'].round(2)
    return dataframe

# Dataframe de los precios de las acciones
historical = [get_price(symbol) for symbol in list_stock]
stocks = pd.concat(historical, ignore_index=True)

# Procesar el DataFrame de precios de acciones
stocks = process_stock_prices(stocks)

In [100]:
stocks

,Date,Price,Ticker
0,2021-01-04,500.30,ADBE
1,2021-01-05,485.88,ADBE
2,2021-01-06,474.10,ADBE
3,2021-01-07,471.00,ADBE
4,2021-01-08,480.00,ADBE
...,...,...,...
28985,2024-06-25,179.62,GOOGL
28986,2024-06-26,182.63,GOOGL
28987,2024-06-27,184.18,GOOGL
28988,2024-06-28,184.32,GOOGL


In [101]:
stocks.to_csv(r"C:\Users\franc\Desktop\Propios\Scraping\Data\stocks.csv",index = False)

### Extracción del Perfil de Acciones

In [102]:
# Dataframe de información de las acciones
info = [get_profile(symbol) for symbol in list_stock]
profile = pd.concat(info, ignore_index=True)

In [103]:
profile

,Sector,Industry,Ticker
0,Technology,Software - Infrastructure,ADBE
1,Technology,Software - Infrastructure,MSFT
2,Technology,Consumer Electronics,AAPL
3,Technology,Semiconductors,QCOM
4,Technology,Semiconductors,AMD
5,Technology,Semiconductors,NVDA
6,Technology,Software - Application,SHOP
7,Technology,Software - Application,CRM
8,Consumer Cyclical,Travel Services,ABNB
9,Consumer Cyclical,Travel Services,BKNG


In [104]:
profile.to_csv(r"C:\Users\franc\Desktop\Propios\Scraping\Data\profile.csv",index = False)

### Procesar Datos Financieros Seleccionados

In [105]:
# Definir una función para convertir columnas a datetime y llenar NaN con 0
def process_financials(dataframe):
    dataframe['Date'] = pd.to_datetime(dataframe['Date'], format='%m/%d/%Y')
    dataframe.fillna(0, inplace=True)
    return dataframe

# Dataframe de los elementos financieros seleccionados 
bucle = [selections(symbol) for symbol in list_stock]
financials = pd.concat(bucle, ignore_index=True)

# Procesar el DataFrame de financials
financials = process_financials(financials)

In [106]:
import datetime

# Función para verificar si una fecha cae en fin de semana
def es_fin_de_semana(fecha):
    dia_semana = fecha.weekday()
    return dia_semana >= 5

# Función para ajustar las fechas que caen en fin de semana
def ajustar_fecha(df):
    # Iterar sobre las filas del DataFrame
    for index, row in df.iterrows():
        fecha = row['Date']
        # Verificar si la fecha cae en fin de semana
        if es_fin_de_semana(fecha):
            # Sumar un día si es sábado (5)
            if fecha.weekday() == 5:
                fecha -= datetime.timedelta(days=1)
            # Sumar un día si es domingo (6)
            elif fecha.weekday() == 6:
                fecha -= datetime.timedelta(days=2)
            # Actualizar la columna 'Date' con la nueva fecha
            df.at[index, 'Date'] = fecha
    return df

# Aplicar la función de ajuste de fecha a toda la columna 'Date'
financials = ajustar_fecha(financials)

In [107]:
# Fusionar las tablas stocks y financials en función de las columnas Date y Ticker que coinciden
numbers = financials.merge(stocks, left_on=['Date', 'Ticker'], right_on=['Date', 'Ticker'],how = "inner")

### Calculo de Métricas Financieras

In [108]:
# Calcular el Market Cap multiplicando las acciones emitidas por el precio
numbers['Market Cap'] = numbers['Share Issued'] * numbers['Price']

# Calcular el Enterprise Value basado en las condiciones
numbers['Enterprice Value'] = np.where(numbers['Net Debt'] > 0, numbers['Market Cap'] + numbers['Net Debt'] , numbers['Market Cap'] + numbers['Total Debt'])

# Calcular el Price-Earnings Ratio (PER)
numbers['PER'] = np.where(numbers['Net Income'] < 0, 0, numbers['Market Cap'] / numbers['Net Income'])

# Calcular el Enterprise Value to EBIT ratio (EV/EBIT)
numbers['EV/EBIT'] = np.where(numbers['EBIT'] < 0, 0, numbers['Enterprice Value'] / numbers['EBIT'])

# Calcular el Enterprise Value to Free Cash Flow ratio (EV/FCF)
numbers['EV/FCF'] = np.where(numbers['Free Cash Flow'] < 0, 0, numbers['Enterprice Value'] / numbers['Free Cash Flow'])

# Calcular el Margen de Beneficio Neto
numbers['Net Profit Margin'] = np.where(numbers['Revenue'] < 0, 0, numbers['Net Income'] / numbers['Revenue'])

# Calcular el Retorno sobre el Capital Invertido (ROIC)
numbers['ROIC'] = np.where(numbers['Equity'] < 0, 0, np.where(numbers['EBIT'] < 0, 0, np.where(numbers['Net Debt'] > 0, numbers['EBIT'] / (numbers['Equity'] + numbers['Net Debt']) , numbers['EBIT'] / (numbers['Equity'] + numbers['Total Debt']))))

In [109]:
numbers

,Date,Revenue,EBIT,Net Income,Total Debt,Net Debt,Share Issued,Equity,Free Cash Flow,Ticker,Price,Market Cap,Enterprice Value,PER,EV/EBIT,EV/FCF,Net Profit Margin,ROIC
0,2023-11-30,19409000.0,6650000.0,5428000.0,4080000.0,0.0,601000.0,16518000.0,6942000.0,ADBE,620.00,3.726200e+08,3.767000e+08,68.647752,56.646617,54.263901,0.279664,0.322847
1,2022-11-30,17606000.0,6098000.0,4756000.0,4633000.0,0.0,601000.0,14051000.0,7396000.0,ADBE,327.12,1.965991e+08,2.012321e+08,41.337073,32.999692,27.208237,0.270135,0.326376
2,2021-11-30,15785000.0,5802000.0,4822000.0,4673000.0,279000.0,601000.0,14797000.0,6882000.0,ADBE,687.22,4.130192e+08,4.132982e+08,85.653094,71.233750,60.054958,0.305480,0.384850
3,2023-06-30,211915000.0,88523000.0,72361000.0,59965000.0,12533000.0,7432000.0,206223000.0,59475000.0,MSFT,335.13,2.490686e+09,2.503219e+09,34.420284,28.277613,42.088595,0.341462,0.404665
4,2022-06-30,198270000.0,83383000.0,72738000.0,61270000.0,35850000.0,7464000.0,166542000.0,65149000.0,MSFT,252.61,1.885481e+09,1.921331e+09,25.921541,23.042239,29.491336,0.366863,0.411988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2022-12-30,116609000.0,28944000.0,23200000.0,26591000.0,0.0,2614000.0,125713000.0,19044000.0,META,117.92,3.082429e+08,3.348339e+08,13.286331,11.568335,17.582119,0.198955,0.190041
98,2021-12-31,117929000.0,46753000.0,39370000.0,13873000.0,0.0,2741000.0,124879000.0,38993000.0,META,342.32,9.382991e+08,9.521721e+08,23.832845,20.366011,24.419053,0.333845,0.336954
99,2023-12-29,307394000.0,84293000.0,73795000.0,28504000.0,0.0,12460000.0,283379000.0,69495000.0,GOOGL,139.47,1.737796e+09,1.766300e+09,23.548969,20.954293,25.416220,0.240066,0.270271
100,2022-12-30,282836000.0,74842000.0,59972000.0,29679000.0,0.0,12849000.0,256144000.0,60010000.0,GOOGL,86.88,1.116321e+09,1.146000e+09,18.614039,15.312259,19.096819,0.212038,0.261847


In [110]:
numbers.to_csv(r"C:\Users\franc\Desktop\Propios\Scraping\Data\numbers.csv",index = False)